In [6]:
import tensorflow as tf
q=tf.FIFOQueue(2,'int32')
init=q.enqueue_many(([0,10],))
x=q.dequeue()
y=x+1
q_inc=q.enqueue([y])

with tf.Session() as sess:
    init.run()
    for _ in range(5):
        v,_=sess.run([x,q_inc])
        print(v)

0
10
1
11
2


In [3]:
import tensorflow as tf
import numpy as np
import threading
import time

#def MyLoop(coord,worker_id):
#    while not coord.should_stop():
#        if np.random.rand()<0.1:
#            print('stopping from id:{} \n'.format(worker_id))
#            coord.request_stop()
#        else:
#            print('working from id:{} \n'.format(worker_id))
#        time.sleep(1)
#
#coord=tf.train.Coordinator()
#threads=[threading.Thread(target=MyLoop,args=(coord,i,)) for i in range(5)]
#for t in threads:
#    t.start()
#coord.join(threads)

queue=tf.FIFOQueue(100,'float')
enqueue_op=queue.enqueue(tf.random_normal([1]))
qr=tf.train.QueueRunner(queue,[enqueue_op]*5)
tf.train.add_queue_runner(qr)
out_tensor=queue.dequeue()

with tf.Session() as sess:
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    for _ in range(3):
        print(sess.run(out_tensor)[0])
    coord.request_stop()
    coord.join(threads)

-1.9285777
-0.9389898
0.49509484


In [9]:
import tensorflow as tf

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

num_shards=2
instances_per_shard=2
for i in range(num_shards):
    filename=('Records/data.tfrecords-%.5d-of-%.5d' % (i,num_shards))
    writer=tf.python_io.TFRecordWriter(filename)
    for j in range(instances_per_shard):
        example=tf.train.Example(features=tf.train.Features(feature={'i':_int64_feature(i),
                                                                     'j':_int64_feature(j)}))
        writer.write(example.SerializeToString())
    writer.close()


In [10]:
files=tf.train.match_filenames_once('Records/data.tfrecords-*')
filename_queue=tf.train.string_input_producer(files,shuffle=False)

reader=tf.TFRecordReader()
_,serialized_example=reader.read(filename_queue)
features=tf.parse_single_example(serialized_example,features={'i':tf.FixedLenFeature([],tf.int64),
                                                              'j':tf.FixedLenFeature([],tf.int64),})

with tf.Session() as sess:
    tf.local_variables_initializer().run()
    print(sess.run(files))
    
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    for i in range(6):
        print(sess.run([features['i'],features['j']]))
    coord.request_stop()
    coord.join(threads)

[b'Records/data.tfrecords-00001-of-00002'
 b'Records/data.tfrecords-00000-of-00002']
[1, 0]
[1, 1]
[0, 0]
[0, 1]
[1, 0]
[1, 1]


In [12]:
files=tf.train.match_filenames_once('Records/data.tfrecords-*')
filename_queue=tf.train.string_input_producer(files,shuffle=False)

reader=tf.TFRecordReader()
_,serialized_example=reader.read(filename_queue)
features=tf.parse_single_example(serialized_example,features={'i':tf.FixedLenFeature([],tf.int64),
                                                              'j':tf.FixedLenFeature([],tf.int64),})
example,label=features['i'],features['j']
batch_size=3
capacity=1000+3*batch_size

example_batch,label_batch=tf.train.batch([example,label],batch_size=batch_size,capacity=capacity)
#example_batch,label_batch=tf.train.shuffle_batch([example,label],batch_size=batch_size,capacity=capacity,min_after_dequeue=30)

with tf.Session() as sess:
    tf.local_variables_initializer().run()
    coord=tf.train.Coordinator()
    threads=tf.train.start_queue_runners(sess=sess,coord=coord)
    
    for i in range(2):
        cur_exmaple_batch,cur_label_batch=sess.run([example_batch,label_batch])
        print(cur_exmaple_batch,cur_label_batch)
    coord.request_stop()
    coord.join(threads)

[1 1 0] [0 1 0]
[0 1 1] [1 0 1]


In [13]:
import tensorflow as tf

files=tf.train.match_filenames_once('Records/output.tfrecords')
filemane_queue=tf.train.string_input_producer(files,shuffle=False)

reader=tf.TFRecordReader()
_,serialized_example=reader.read(filemane_queue)
features=tf.parse_single_example(serialized_example,features={'image_raw':tf.FixedLenFeature([],tf.string),
                                                              'pixels':tf.FixedLenFeature([],tf.int64),
                                                              'label':tf.FixedLenFeature([],tf.int64)})
decoded_images=tf.decode_raw(features['image_raw'],tf.uint8)
retyped_images=tf.cast(decoded_images,tf.float32)
labels=tf.cast(features['label'],tf.int32)

images=tf.reshape(retyped_images,[784])

min_after_dequeue=1000
batch_size=100
capacity=min_after_dequeue+3*batch_size

image_batch,label_batch=tf.train.shuffle_batch([images,labels],batch_size=batch_size,capacity=capacity,min_after_dequeue=min_after_dequeue)

def inference(input_tensor,weights1,biases1,weights2,biases2):
    layer1=tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
    return tf.matmul(layer1,weights2)+biases2

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500
REGULARAZTION_RATE = 0.0001   
TRAINING_STEPS = 5000        

weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE, LAYER1_NODE], stddev=0.1))
biases1 = tf.Variable(tf.constant(0.1, shape=[LAYER1_NODE]))

weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE, OUTPUT_NODE], stddev=0.1))
biases2 = tf.Variable(tf.constant(0.1, shape=[OUTPUT_NODE]))

y = inference(image_batch, weights1, biases1, weights2, biases2)

cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=label_batch)
cross_entropy_mean = tf.reduce_mean(cross_entropy)
    
regularizer = tf.contrib.layers.l2_regularizer(REGULARAZTION_RATE)
regularaztion = regularizer(weights1) + regularizer(weights2)
loss = cross_entropy_mean + regularaztion

train_step = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    tf.local_variables_initializer().run()
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    for i in range(TRAINING_STEPS):
        if i % 1000 == 0:
            print("After %d training step(s), loss is %g " % (i, sess.run(loss)))
        sess.run(train_step) 
    coord.request_stop()
    coord.join(threads) 

After 0 training step(s), loss is 402.868 
After 1000 training step(s), loss is 6.63943 
After 2000 training step(s), loss is 6.50159 
After 3000 training step(s), loss is 7.24043 
After 4000 training step(s), loss is 6.47796 
